# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

---

First, let's start Ray…

In [1]:
import logging
import time
import ray
import random
from random import randint

In [2]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2021-12-26_14-26-21_967351_91263/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-26_14-26-21_967351_91263/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-26_14-26-21_967351_91263',
 'metrics_export_port': 63402,
 'node_id': '44741adff787ed6d54c064e54f40b12551efd33c553fa01580dda5b6'}

## 3. Remote Class as a Stateful Actor Pattern

To start, we'll define a class and use the decorator:

Let's use Python class and convert that to a remote Actor class and create multiple actor handle instances associated with a distinct attributes, such as a name, age, goals scored, etc

In [3]:
@ray.remote
class GoalsScored:
    def __init__ (self, player, age) -> None:
        self._goals = 0
        self._player = player
        self._age = age

    def score (self, goal) -> object:
        self._goals += goal
        return self._goals
       
    def player(self) -> str:
        return self._player
    
    # Any method of the actor can return multiple object refs.
    @ray.method(num_returns=3)
    def stats(self) -> object:
        return self._player, self._age, self._goals

Define three Actors: Rolando, Neymar, Messi

In [4]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo", randint(18, 35))
neymar = GoalsScored.remote("Neymar", randint(18, 35))
messi = GoalsScored.remote("Messi", randint(18, 35))

CPU times: user 32.4 ms, sys: 11.4 ms, total: 43.8 ms
Wall time: 41.8 ms


Update the scores for each player

In [5]:
%%time

ronaldo.score.remote(randint(1, 7))
neymar.score.remote(randint(1, 7))
messi.score.remote(randint(1, 7))

CPU times: user 539 µs, sys: 93 µs, total: 632 µs
Wall time: 568 µs


ObjectRef(32cccd03c567a254362b7a44e0b31d4c0a8545b90100000001000000)

Again, use list comprehension to iterate over each Actor handle instances, along with object_ref for their goals scores, maintained by each distinct actor.

In [6]:
def print_stats():
    for ref in [ronaldo, neymar, messi]:
        print(f"Player: {ray.get(ref.stats.remote())}")

In [7]:
print_stats()

Player: ['Ronaldo', 24, 1]
Player: ['Neymar', 20, 4]
Player: ['Messi', 24, 3]


Add three goals for for Neymar

In [8]:
[neymar.score.remote(goal) for goal in range(3)]
print_stats()

Player: ['Ronaldo', 24, 1]
Player: ['Neymar', 20, 7]
Player: ['Messi', 24, 3]


## Tree of Actors Pattern

A common pattern used in Ray libraries ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html) and [Ray Train](https://docs.ray.io/en/latest/train/train.html)) to train models in a parallel or distributed manners.

In this common pattern, tree of actors, a collection of workers as actors, are managed by a supervisor. For example, you want to train multiple models at the same time, while being able to checkpoint/inspect its state.

<img src="https://docs.ray.io/en/latest/_images/tree-of-actors.svg" width="50%" height="40%">

Let's implement a simple exampel to illustrate this pattern.

In [9]:
STATES = ["RUNNING", "DONE"]

class Model:

    def __init__(self, m:str):
        self._model = m

    def train(self):
        # do some training here
        time.sleep(1)
    
def model_factory(m: str):
    return Model(m)

In [10]:
@ray.remote
class Worker(object):
    def __init__(self, m:str):
        self._model = m
        
    def state(self) -> str:
        return random.choice(STATES)
    
    def work(self) -> None:
        model_factory(self._model).train()
         
@ray.remote
class Supervisor:
    def __init__(self):
        self.workers = [Worker.remote(name) for name in ["lr", "cl", "lrn"]]
                        
    def work(self):
        [w.work.remote() for w in self.workers]
        
    def terminate(self):
        [ray.kill(w) for w in self.workers]
        
    def state(self):
        return ray.get([w.state.remote() for w in self.workers])

In [11]:
# Create a Actor instance for supervisor
sup = Supervisor.remote()

# Launch remote actors as workers
sup.work.remote()

ObjectRef(24eed4584329c19afa564abfe3d48d6e9ea69ee90100000001000000)

In [12]:
# check their status
while True:
    states = ray.get(sup.state.remote())
    print(states)
    result = all('DONE' == e for e in states)
    if result:
        # Note: Actor processes will be terminated automatically when the initial actor handle goes out of scope in Python. 
        # If we create an actor with actor_handle = ActorClass.remote(), then when actor_handle goes out of scope and is destructed, 
        # the actor process will be terminated. Note that this only applies to the original actor handle created for the actor 
        # and not to subsequent actor handles created by passing the actor handle to other tasks.
        
        # kill supervisors all worker manually, only for illustrtation and demo
        sup.terminate.remote()

        # kill the supervisor manually, only for illustration and demo
        ray.kill(sup)
        break

['RUNNING', 'DONE', 'RUNNING']
['DONE', 'DONE', 'RUNNING']
['RUNNING', 'RUNNING', 'DONE']
['RUNNING', 'RUNNING', 'DONE']
['RUNNING', 'DONE', 'DONE']
['DONE', 'DONE', 'RUNNING']
['DONE', 'DONE', 'RUNNING']
['RUNNING', 'DONE', 'RUNNING']
['DONE', 'DONE', 'RUNNING']
['RUNNING', 'DONE', 'RUNNING']
['RUNNING', 'DONE', 'DONE']
['RUNNING', 'RUNNING', 'RUNNING']
['DONE', 'RUNNING', 'DONE']
['DONE', 'DONE', 'RUNNING']
['RUNNING', 'DONE', 'DONE']
['DONE', 'RUNNING', 'RUNNING']
['DONE', 'DONE', 'DONE']


Finally, shutdown Ray

In [13]:
ray.shutdown()

---
## References

 * [A Universal Modular Actor Formalism for Artificial Intelligence](https://www.ijcai.org/Proceedings/73/Papers/027B.pdf)  
Carl Hewitt, Peter Bishop, Richard Steiger  *IJCAI* (1973)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)
 * [Advanced Patterns and Anti-Patterns in Ray](https://docs.ray.io/en/latest/ray-design-patterns/index.htmlhttps://docs.ray.io/en/latest/ray-design-patterns/index.html)